<a href="https://colab.research.google.com/github/aaronc09/peds_lupus_flare_prediction_ML_gene-expression/blob/main/2_FEATURE_SELECTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import pickle

drive_path = '/content/drive/MyDrive/lupus_final_df.pkl'
lupus_final_df = pd.read_pickle(drive_path)

lupus_final_df.shape

Mounted at /content/drive


(440, 43921)

## 1. Univariate linear regression (OLS): gene-expression vs SLEDAI

In [2]:
import statsmodels.api as sm

# Copy previously prepared dataFrame containing data from lupus samples only
lup = lupus_final_df.copy()

# Loop through each gene-expression probe
ilmn_cols = [col for col in lup.columns if col.startswith("ILMN_")]

# Create a list to store regression result per gene-expression
results = []

# Loop through each gene-expression column
for col in ilmn_cols:
    valid = lup[[col, "sledai_current"]].dropna()
    if len(valid) < 3:
      continue

    # x=gene-expression values & y= SLEDAI scores
    x = valid[col]
    y = valid["sledai_current"]

    # Calculate standard deviations for both
    std_x = x.std(ddof=0)
    std_y = y.std(ddof=0)

    # Exclude gene-expression if values under that gene-expression have no variation (std is zero)
    # Exclude SLEDAI scores that have no variation (std is zero)
    if std_x == 0 or std_y == 0:
        continue

    # Standardize both gene-expression values (x) and SLEDAI scores (y)
    x_z = (x - x.mean()) / std_x
    y_z = (y - y.mean()) / std_y

    # Run an Ordinary Least Squares (OLS) regression model:
    # Standardized SLEDAI scores vs standardized gene-expression values, one at a time
    # Then extract the gene-expression's beta, t-statistic, and p-value + summary stats
    X = sm.add_constant(x_z)
    model = sm.OLS(y_z, X).fit()
    predictor = x_z.name
    beta = model.params.loc[predictor]
    t_val = model.tvalues.loc[predictor]
    p_val = model.pvalues.loc[predictor]
    res = {
        "gene": col,
        "beta": beta,
        "t_val": t_val,
        "p_val": p_val,
        "n": len(valid),
        "mean_ILMN": x.mean(),
        "sd_ILMN": std_x,
        "mean_sledai": y.mean(),
        "sd_sledai": std_y,
    }

    # Estimated change in raw SLEDAI score per 1 standard deviation increase in gene expression value
    sledai_score_change = beta * std_y

    # Add SLEDAI change estimates- key/value to the result dictionary
    res["sledai_score_change"] = sledai_score_change

    # Add the SLEDAI change estimates to the "results" list
    results.append(res)

# Convert results to a pandas dataframe with the name "gene_sledai_df"
gene_sledai_df = pd.DataFrame(results)

# Convert sledai_score_change values in the "gene_sledai_df" to absolute values
gene_sledai_df["abs_sledai_score_change"] = gene_sledai_df["sledai_score_change"].abs()

In [3]:
# 43,799 gene-expressions processesed through univariate regression vs SLEDAI
gene_sledai_df.shape

(43799, 11)

In [4]:
# First 5 sample results
gene_sledai_df.head()

,gene,beta,t_val,p_val,n,mean_ILMN,sd_ILMN,mean_sledai,sd_sledai,sledai_score_change,abs_sledai_score_change
0,ILMN_1343291,-0.221413,-4.751775,0.000003,440,13.747101,0.276101,7.956818,6.241026,-1.381846,1.381846
1,ILMN_1343295,0.181720,3.867514,0.000127,440,10.743433,0.399282,7.956818,6.241026,1.134120,1.134120
2,ILMN_1651199,0.027143,0.568266,0.570146,440,3.328777,0.056374,7.956818,6.241026,0.169399,0.169399
3,ILMN_1651209,0.021904,0.458531,0.646798,440,3.348926,0.152006,7.956818,6.241026,0.136705,0.136705
4,ILMN_1651210,0.004022,0.084171,0.932959,440,3.386683,0.202774,7.956818,6.241026,0.025100,0.025100


## 2. Welch's T-test: lupus sample vs healthy control sample gene-expressions




In [5]:
#T-test for SLE vs healthy control gene-expression values
#obtained from 924 Lupus subject office visits and 72 non-SLE healthy control visits

drive_path = '/content/drive/MyDrive/full_df.pkl'
full_df=pd.read_pickle(drive_path)

In [6]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind

# Load the full dataframe which includes data from both lupus and healthy control samples
drive_path = '/content/drive/MyDrive/full_df.pkl'
full_df = pd.read_pickle(drive_path)

# Split dataset into 2 different groups- lupus and healthy control
sle_df = full_df[full_df['disease state'] == 'SLE']
healthy_df = full_df[full_df['disease state'] == 'Healthy']

# Loop through and create a list of all gene-expression column names
ilmn_cols_ttest = [col for col in full_df.columns if col.startswith('ILMN_')]

# Create an empty list to store the result of each gene’s t-test output for each gene
results_ttest = []

# If a gene-expression in nearly constant within either group, Welch's t-test is not informative
EPS = 1e-12

# Loop through each gene-expression values for each group
for col in ilmn_cols_ttest:
    # selects from lupus samples
    sle_values = sle_df[col].dropna().to_numpy(dtype=float)
    # selects from healthy samples
    healthy_values = healthy_df[col].dropna().to_numpy(dtype=float)

    # A t-test needs at least 2 values per group
    if len(sle_values) > 1 and len(healthy_values) > 1:
        # calculate mean expression in each group
        mean_sle = float(np.mean(sle_values))
        mean_healthy = float(np.mean(healthy_values))

        #calculat standrad deviation (SD) in each group
        std_sle = float(np.std(sle_values, ddof=1))
        std_healthy = float(np.std(healthy_values, ddof=1))

        # Skip gene-expressions with zero or close to zero variance in either group
        if std_sle < EPS or std_healthy < EPS:
            continue

        # Apply Welch's t-test
        t_stat, p_val = ttest_ind(sle_values, healthy_values, equal_var=False)

        results_ttest.append({
            "gene": col,
            "t_statistic": float(t_stat),
            "p_val": float(p_val),
            "mean_sle": mean_sle,
            "std_sle": std_sle,
            "mean_healthy": mean_healthy,
            "std_healthy": std_healthy,
            "ilmn_diff": mean_healthy - mean_sle
        })

lupus_v_control_df = pd.DataFrame(results_ttest)

In [7]:
lupus_v_control_df.head()

,gene,t_statistic,p_val,mean_sle,std_sle,mean_healthy,std_healthy,ilmn_diff
0,ILMN_1343291,-2.652551,0.009433,13.761880,0.268199,13.830907,0.207732,0.069027
1,ILMN_1343295,1.701279,0.092663,10.689609,0.408287,10.604291,0.409982,-0.085318
2,ILMN_1651199,1.758586,0.079766,3.330999,0.073447,3.325052,0.020077,-0.005947
3,ILMN_1651209,0.356824,0.721950,3.343862,0.125600,3.340318,0.076640,-0.003544
4,ILMN_1651210,1.381737,0.170425,3.378239,0.191229,3.353004,0.145483,-0.025235


In [8]:
lupus_v_control_df.shape

(39491, 8)

## 3. Welch's t-test: lupus pre-flare vs lupus non-pre-flare samples

In [9]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind

# preflare state = "True" & nonpreflare state = "False"
preflare = lup[lup["preflare_bool"] == True]
nonpreflare = lup[lup["preflare_bool"] == False]

# Loop through and create a list of all gene-expression column names
ilmn_cols_ttest = [col for col in lup.columns if col.startswith("ILMN_")]

# Create an empty list to store the result of each gene’s t-test output for each gene
results_ttest = []

# If a gene-expression value has almost no variation (SD is near zero), t-test becomes unreliable
# So these gene-expression values will be skipped later on
EPS = 1e-12

# Loop through each gene-expression values for each group
for col in ilmn_cols_ttest:
    # selects from lupus preflare samples
    preflare_values = preflare[col].dropna().to_numpy(dtype=float)
    # selects from lupus nonpreflare samples
    nonpreflare_values = nonpreflare[col].dropna().to_numpy(dtype=float)

    # A t-test needs at least 2 values per group
    if len(preflare_values) > 1 and len(nonpreflare_values) > 1:
        # calculate mean expression in each group
        mean_preflare = float(np.mean(preflare_values))
        mean_nonpreflare = float(np.mean(nonpreflare_values))

        #calculat standrad deviation (SD) in each group
        std_preflare = float(np.std(preflare_values, ddof=1))
        std_nonpreflare = float(np.std(nonpreflare_values, ddof=1))

        # Skip gene-expressions that are (almost) constant in either group
        if std_preflare < EPS or std_nonpreflare < EPS:
            continue

        # Apply Welch’s t-test
        t_stat, p_val = ttest_ind(preflare_values, nonpreflare_values, equal_var=False)

        results_ttest.append({
            "gene": col,
            "t_statistic": float(t_stat),
            "p_val": float(p_val),
            "mean_preflare": mean_preflare,
            "std_preflare": std_preflare,
            "mean_nonpreflare": mean_nonpreflare,
            "std_nonpreflare": std_nonpreflare,
            "ilmn_diff": mean_preflare - mean_nonpreflare
        })

preflare_v_nonpreflare_df = pd.DataFrame(results_ttest)
preflare_v_nonpreflare_df.head()

,gene,t_statistic,p_val,mean_preflare,std_preflare,mean_nonpreflare,std_nonpreflare,ilmn_diff
0,ILMN_1343291,0.357367,0.721521,13.757247,0.262767,13.745083,0.279351,0.012163
1,ILMN_1343295,-2.050909,0.042494,10.666289,0.339300,10.758778,0.409377,-0.092489
2,ILMN_1651199,0.331273,0.741061,3.330628,0.051165,3.328409,0.057487,0.002219
3,ILMN_1651209,0.371743,0.710904,3.355478,0.167843,3.347623,0.149082,0.007855
4,ILMN_1651210,-1.074200,0.284755,3.367310,0.159076,3.390536,0.210625,-0.023227


In [10]:
preflare_v_nonpreflare_df.head()

,gene,t_statistic,p_val,mean_preflare,std_preflare,mean_nonpreflare,std_nonpreflare,ilmn_diff
0,ILMN_1343291,0.357367,0.721521,13.757247,0.262767,13.745083,0.279351,0.012163
1,ILMN_1343295,-2.050909,0.042494,10.666289,0.339300,10.758778,0.409377,-0.092489
2,ILMN_1651199,0.331273,0.741061,3.330628,0.051165,3.328409,0.057487,0.002219
3,ILMN_1651209,0.371743,0.710904,3.355478,0.167843,3.347623,0.149082,0.007855
4,ILMN_1651210,-1.074200,0.284755,3.367310,0.159076,3.390536,0.210625,-0.023227


In [11]:
preflare_v_nonpreflare_df.shape

(40002, 8)

In [12]:
preflare_v_nonpreflare_df.head()

,gene,t_statistic,p_val,mean_preflare,std_preflare,mean_nonpreflare,std_nonpreflare,ilmn_diff
0,ILMN_1343291,0.357367,0.721521,13.757247,0.262767,13.745083,0.279351,0.012163
1,ILMN_1343295,-2.050909,0.042494,10.666289,0.339300,10.758778,0.409377,-0.092489
2,ILMN_1651199,0.331273,0.741061,3.330628,0.051165,3.328409,0.057487,0.002219
3,ILMN_1651209,0.371743,0.710904,3.355478,0.167843,3.347623,0.149082,0.007855
4,ILMN_1651210,-1.074200,0.284755,3.367310,0.159076,3.390536,0.210625,-0.023227


## >> extract gene-expression features from the intersect of the above 3 analyses

In [13]:
#1. Extracting features from the "Univariate linear regression (OLS): gene-expression vs SLEDAI" analysis
sledai_extracted = set(gene_sledai_df[(gene_sledai_df["p_val"] < 0.2)]["gene"])

#2. Extracting features from the "Welch's T-test: lupus subjects vs healthy control subjects" analysis
lupus_extracted = set(lupus_v_control_df[(lupus_v_control_df["p_val"] < 0.2)]["gene"])

#3. Extracting features from the "Welch's t-test: lupus pre-flare vs lupus non-pre-flare samples" analysis
flare_extracted = set(preflare_v_nonpreflare_df[(preflare_v_nonpreflare_df["p_val"] < 0.2)]["gene"])


#>> Final extraction of gene-expression features meeting p < 0.2 in all 3 analyses
features_selected = sledai_extracted.intersection(lupus_extracted).intersection(flare_extracted)

In [14]:
#1. Number of gene-expressions selected from Univariate linear regression (OLS): gene-expression vs SLEDAI
len(sledai_extracted)

13119

In [15]:
#2. Number of gene-expressions selected from Welch's T-test: lupus subjects vs healthy control subjects
len(lupus_extracted)

26132

In [16]:
#3. Number of gene-expressions selected from: Welch's t-test: lupus pre-flare vs lupus non-pre-flare samples
len(flare_extracted)

8468

In [17]:
# >> number of gene-expression features from intersect from the 3 feature selection analyses
len(features_selected)

1979

## collinearity analysis to identify redundant gene-expression features for removal if found


In [18]:
# Convert the extracted gene-expressions from intersect (formed from the previous cell) into a list
selected_ilmn_cols = list(features_selected)

# Perform collinearity analaysis
correlation_matrix = lupus_final_df[selected_ilmn_cols].corr()

ilmn_columns_from_corr = correlation_matrix.columns

redundant_pairs = []
for i in range(len(ilmn_columns_from_corr)):
    for j in range(i + 1, len(ilmn_columns_from_corr)):
        col1 = ilmn_columns_from_corr[i]
        col2 = ilmn_columns_from_corr[j]
        if abs(correlation_matrix.loc[col1, col2]) > 0.95:
            #if two columns are highly correlated, one to be dropped.
            redundant_pairs.append((col1, col2))

# Number of redundant pairs identified
len(redundant_pairs)

0

In [19]:
# Remove redundant gene-expressions if found
features_selected = sledai_extracted.intersection(lupus_extracted).intersection(flare_extracted)
for col1, col2 in sorted(redundant_pairs):
    if col1 in features_selected and col2 in features_selected:
        print("Removing redundant feature: " + col1)
        features_selected.remove(col1)

In [20]:
# The final number of gene-expression features selected
len(features_selected)

1979

In [21]:
extracted_cols_path = '/content/drive/MyDrive/features_selected.pkl'
with open(extracted_cols_path, 'wb') as f:
    pickle.dump(features_selected, f)